In [1]:
!pip install tabpfn
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
from captum.attr import DeepLift
import torch

In [3]:
dataset = pd.read_csv("https://raw.github.com/mattdelhey/kaggle-titanic/master/Data/train.csv")

# impute age with median
dataset['age'].fillna(dataset['age'].median(),inplace=True)

# impute embarked with forward fill
dataset['embarked'].fillna(method ='ffill', inplace=True)

# drop name, ticket, cabin column
dataset.drop(["name","ticket","cabin"],axis=1,inplace=True)

# convert age as int
dataset['age'] = dataset['age'].astype(int)

# one hot encode Embarked column and drop it from the original df
encoded_df = pd.get_dummies(dataset['embarked'], prefix='embarked')
df_encoded = pd.concat([dataset, encoded_df], axis=1)
df_encoded = df_encoded.drop('embarked', axis=1)

# one hot encode Class column and drop it from the original df
encoded_df = pd.get_dummies(df_encoded['pclass'], prefix='pclass')
df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
df_encoded = df_encoded.drop('pclass', axis=1)

# one hot encode Sex column and drop it from the original df
encoded_df = pd.get_dummies(df_encoded['sex'], prefix='sex')
df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
df_encoded = df_encoded.drop('sex', axis=1)

df_encoded.head()
print(df_encoded.columns)

Index(['survived', 'age', 'sibsp', 'parch', 'fare', 'embarked_C', 'embarked_Q',
       'embarked_S', 'pclass_1', 'pclass_2', 'pclass_3', 'sex_female',
       'sex_male'],
      dtype='object')


In [4]:
X = df_encoded.drop('survived', axis=1)  # Features
y = df_encoded['survived']  # Target variable

# print(type(X))
# print(type(y))

# Split the dataset into training and testing sets
#train, test = train_test_split(df_encoded, test_size=0.2, stratify=dataset["survived"], random_state=42)
# val, test  = train_test_split(tmp, test_size=0.5, stratify=tmp["survived"], random_state=42)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print(type(x_train_ss))

# yaha changes kar ri thi mai
# X = df_encoded.drop('survived', axis=1)  # Features
# y = df_encoded['survived']  # Target variable

In [5]:
# is model pe lagana h deeplift
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier

classifier = TabPFNClassifier(device='cpu')
classifier.fit(x_train, y_train)
y_eval, p_eval = classifier.predict(x_test, return_winning_probability=True)

print('Accuracy', accuracy_score(y_test, y_eval))

We have to download the TabPFN, as there is no checkpoint at  /usr/local/lib/python3.10/dist-packages/tabpfn/models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt
It has about 100MB, so this might take a moment.
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen 

Accuracy 0.8156424581005587


In [6]:
x_train_t = torch.tensor(x_train.values, dtype=torch.float32)
x_test_t = torch.tensor(x_test.values, dtype=torch.float32)
y_train_t = torch.tensor(y_train.values, dtype=torch.float32)
y_test_t = torch.tensor(y_test.values, dtype=torch.float32)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# ye implement ni kiya h abhi.. draft h bas aise hi
dl = DeepLift(classifier)
x_train_attr = dl.attribute(x_train_t)
y_train_attr = dl.attribute(y_train_t)

x_test_attr = dl.attribute(x_test_t)
y_test_attr = dl.attribute(y_test_t)

print(x_train_attr, y_train_attr, x_test_attr, y_test_attr)

/usr/local/lib/python3.10/dist-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/captum/attr/_core/deep_lift.py:304: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(


AttributeError: ignored